In [ ]:
import torch
import h5py
import numpy as np
import networkx as nx
import scipy.sparse as sp
import pandas as pd
import torch.nn.functional as F
import torch.nn as nn
import torch.nn.init as init
import torch.nn.functional as F
import torch.optim as optim
from torch_geometric.nn import GCNConv,GATConv,GATv2Conv,SAGEConv,TransformerConv,Node2Vec
from torch_geometric.data import Data
from torch_geometric.utils import degree
from torch_geometric.utils import negative_sampling
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

In [ ]:
from operator import xor
torch.set_default_tensor_type(torch.DoubleTensor)
class My_GCN(nn.Module):
    def __init__(self, graph):
        super(My_GCN, self).__init__()
        self.graph = graph
        self.num_nodes = graph.num_nodes
        self.edges = graph["edge_index"].to(device)
        self.in_c = 128
        self.hid_c = 256
        self.out_c = 128

        self.dropout = 0
        self.linear = nn.Linear(self.num_nodes, self.in_c)
        self.gcn1 = GCNConv(self.in_c, self.hid_c)
        self.gcn2 = GCNConv(self.hid_c, self.out_c)

    def forward(self):
        ini_feature ,edge_index = self.graph["x"].to(device),self.graph["edge_index"].to(device)
        x = torch.tanh(self.linear(ini_feature))
        x = torch.tanh(self.gcn1(x, edge_index))
        x = F.dropout(x,self.dropout)
        x = self.gcn2(x, edge_index)
        x = F.dropout(x,self.dropout)

        return x 

    def loss(self, after_mapping):
        indices1 = self.graph["edge_index"]
        source_feats1 = after_mapping[indices1[0].type(torch.long)].to(device)
        target_feats1 = after_mapping[indices1[1].type(torch.long)].to(device)
        num_sampled= int(self.graph.num_edges/2)
        negative_deges = negative_sampling(edge_index=self.edges,num_nodes=self.graph.num_nodes,num_neg_samples=num_sampled)
        negative_src = torch.tensor(negative_deges[0])
        negative_trg = torch.tensor(negative_deges[1])
        source_feats3 = after_mapping[negative_src.type(torch.long)].to(device)
        target_feats3 = after_mapping[negative_trg.type(torch.long)].to(device)

        dots_1= torch.sum(torch.mul(source_feats1,target_feats1), dim=1)
        dots1_loss = torch.mean(-1.0 * F.logsigmoid(dots_1))
        dots_3= torch.sum(torch.mul(source_feats3,target_feats3), dim=1)
        dots3_loss = torch.mean(-1.0 * torch.log(1.000001 - torch.sigmoid(dots_3)))

        mapping_loss = dots1_loss + dots3_loss
        return mapping_loss, dots1_loss, dots3_loss

class DGCN(nn.Module):
    def __init__(self, sim_graph,graph,dropout):
        super(DGCN, self).__init__()
        self.sim_graph = sim_graph
        self.graph = graph
        self.num_nodes = graph.num_nodes
        self.edges = sim_graph["edge_index"]
        self.node_degree = degree(self.graph["edge_index"][0]).t()
        self.in_c = 128
        self.hid_c = 128*2
        self.out_c = 64*2
        self.dropout = dropout

        self.linear = nn.Linear(self.num_nodes, self.in_c)
        self.gcn1 = GCNConv(self.in_c, self.hid_c)
        self.gcn2 = GCNConv(self.hid_c,self.out_c)
        self.gcn3 = GCNConv(self.out_c, self.hid_c)
        self.gcn4 = GCNConv(self.hid_c, self.out_c)
        self.dcn = GCNConv(self.out_c*2,self.out_c)
        self.mlp = nn.Linear(self.out_c*2, self.out_c*2)

    def forward(self):
        ini_feature ,edge_index,motif_W = self.graph["x"],self.graph["edge_index"],self.graph["edge_attr"]
        edge_index_sim= self.sim_graph["edge_index"]
        x = torch.tanh(self.linear(ini_feature.to(torch.double)))
        x1 = torch.tanh(self.gcn1(x, edge_index_sim))
        x1 = F.dropout(x1,self.dropout)
        x2 = torch.tanh(self.gcn2(x1, edge_index_sim))
        x2 = F.dropout(x2,self.dropout)
        x3 = torch.tanh(self.gcn3(x2, edge_index,edge_weight=motif_W))
        x3 = F.dropout(x3,self.dropout)
        x4 = torch.tanh(self.gcn4(x3, edge_index,edge_weight=motif_W))
        x4 = F.dropout(x4,self.dropout)
        x_out = torch.cat((x2,x4),1)
        x_out_nc = self.mlp(x_out)
        return x_out_nc


    def loss(self, after_mapping):
        indices1 = self.graph["edge_index"]
        source_feats1 = after_mapping[indices1[0].type(torch.long)].to(device)
        target_feats1 = after_mapping[indices1[1].type(torch.long)].to(device)
        num_sampled= int(self.graph.num_edges/2)
        negative_deges = negative_sampling(edge_index=self.edges,num_nodes=self.graph.num_nodes,num_neg_samples=num_sampled)
        negative_src = torch.tensor(negative_deges[0])
        negative_trg = torch.tensor(negative_deges[1])
        source_feats3 = after_mapping[negative_src.type(torch.long)].to(device)
        target_feats3 = after_mapping[negative_trg.type(torch.long)].to(device)

        dots_1= torch.sum(torch.mul(source_feats1,target_feats1), dim=1)
        dots1_loss = torch.mean(-1.0 * F.logsigmoid(dots_1))
        dots_3= torch.sum(torch.mul(source_feats3,target_feats3), dim=1)
        dots3_loss = torch.mean(-1.0 * torch.log(1.00001 - torch.sigmoid(dots_3)))

        mapping_loss = dots1_loss + dots3_loss
        return mapping_loss, dots1_loss, dots3_loss

class DGCN1(nn.Module):
    def __init__(self, sim_graph,graph,dropout):
        super(DGCN1, self).__init__()
        self.sim_graph = sim_graph
        self.graph = graph
        self.num_nodes = graph.num_nodes
        self.edges = sim_graph["edge_index"]
        self.node_degree = degree(self.graph["edge_index"][0]).t()
        self.in_c = 128
        self.hid_c = 256
        self.out_c = 128
        self.dropout = dropout

        self.linear = nn.Linear(self.num_nodes, self.in_c)
        self.gcn1 = GCNConv(self.in_c, self.hid_c)
        self.gcn2 = GCNConv(self.hid_c,self.out_c)
        self.gcn3 = GCNConv(self.out_c, self.hid_c)
        self.gcn4 = GCNConv(self.hid_c, self.out_c)
        self.dcn = GCNConv(self.out_c*2,self.out_c)
        self.mlp = nn.Linear(self.out_c*2, self.out_c*2)

    def forward(self):
        ini_feature ,edge_index,motif_W = self.graph["x"],self.graph["edge_index"],self.graph["edge_attr"]
        edge_index_sim= self.sim_graph["edge_index"]
        x = torch.tanh(self.linear(ini_feature.to(torch.double)))
        x1 = torch.tanh(self.gcn1(x, edge_index_sim))
        x1 = F.dropout(x1,self.dropout)
        x2 = torch.tanh(self.gcn2(x1, edge_index_sim))
        x2 = F.dropout(x2,self.dropout)
        x3 = torch.tanh(self.gcn3(x2, edge_index,edge_weight=motif_W))
        x3 = F.dropout(x3,self.dropout)
        x4 = torch.tanh(self.gcn4(x3, edge_index,edge_weight=motif_W))
        x4 = F.dropout(x4,self.dropout)
        x_out = torch.cat((x2,x4),1)
        x_out_nc = self.mlp(x_out)
        return x_out_nc


    def loss(self, after_mapping):
        indices1 = self.graph["edge_index"]
        source_feats1 = after_mapping[indices1[0].type(torch.long)].to(device)
        target_feats1 = after_mapping[indices1[1].type(torch.long)].to(device)
        num_sampled= int(self.graph.num_edges/2)
        negative_deges = negative_sampling(edge_index=self.edges,num_nodes=self.graph.num_nodes,num_neg_samples=num_sampled)
        negative_src = torch.tensor(negative_deges[0])
        negative_trg = torch.tensor(negative_deges[1])
        source_feats3 = after_mapping[negative_src.type(torch.long)].to(device)
        target_feats3 = after_mapping[negative_trg.type(torch.long)].to(device)

        dots_1= torch.sum(torch.mul(source_feats1,target_feats1), dim=1)
        dots1_loss = torch.mean(-1.0 * F.logsigmoid(dots_1))
        dots_3= torch.sum(torch.mul(source_feats3,target_feats3), dim=1)
        dots3_loss = torch.mean(-1.0 * torch.log(1.00001 - torch.sigmoid(dots_3)))

        mapping_loss = dots1_loss + dots3_loss
        return mapping_loss, dots1_loss, dots3_loss

class MD_GCN(nn.Module):
    def __init__(self, sim_graph,graph,dropout):
        super(MD_GCN, self).__init__()
        self.sim_graph = sim_graph
        self.graph = graph
        self.num_nodes = graph.num_nodes
        self.edges = sim_graph["edge_index"]
        self.node_degree = degree(self.graph["edge_index"][0]).t()
        self.in_c = 128
        self.hid_c = 256
        self.out_c = 128
        self.dropout = dropout

        self.linear = nn.Linear(self.num_nodes, self.in_c)
        self.gcn1 = GCNConv(self.in_c, self.hid_c)
        self.gcn2 = GCNConv(self.hid_c,self.out_c)
        self.gcn3 = GCNConv(self.out_c, self.hid_c)
        self.gcn4 = GCNConv(self.hid_c, self.out_c)
        self.dcn = GCNConv(self.out_c*2,self.out_c)
        self.mlp = nn.Linear(self.out_c*2, self.out_c*2)

    def forward(self):
        ini_feature ,edge_index,motif_W = self.graph["x"],self.graph["edge_index"],self.graph["edge_attr"]
        edge_index_sim= self.sim_graph["edge_index"]
        x = torch.tanh(self.linear(ini_feature.to(torch.double)))
        x1 = torch.tanh(self.gcn1(x, edge_index_sim))
        x1 = F.dropout(x1,self.dropout)
        x2 = torch.tanh(self.gcn2(x1, edge_index_sim))
        x2 = F.dropout(x2,self.dropout)
        x3 = torch.tanh(self.gcn3(x2, edge_index,edge_weight=motif_W))
        x3 = F.dropout(x3,self.dropout)
        x4 = torch.tanh(self.gcn4(x3, edge_index,edge_weight=motif_W))
        x4 = F.dropout(x4,self.dropout)
        x_out = torch.cat((x2,x4),1)
        x_out_nc = self.mlp(x_out)
        return x_out_nc


    def loss(self, after_mapping):
        indices1 = self.graph["edge_index"]
        source_feats1 = after_mapping[indices1[0].type(torch.long)].to(device)
        target_feats1 = after_mapping[indices1[1].type(torch.long)].to(device)
        num_sampled= int(self.graph.num_edges/2)
        negative_deges = negative_sampling(edge_index=self.edges,num_nodes=self.graph.num_nodes,num_neg_samples=num_sampled)
        negative_src = torch.tensor(negative_deges[0])
        negative_trg = torch.tensor(negative_deges[1])
        source_feats3 = after_mapping[negative_src.type(torch.long)].to(device)
        target_feats3 = after_mapping[negative_trg.type(torch.long)].to(device)

        dots_1= torch.sum(torch.mul(source_feats1,target_feats1), dim=1)
        dots1_loss = torch.mean(-1.0 * F.logsigmoid(dots_1))
        dots_3= torch.sum(torch.mul(source_feats3,target_feats3), dim=1)
        dots3_loss = torch.mean(-1.0 * torch.log(1.00001 - torch.sigmoid(dots_3)))

        mapping_loss = dots1_loss + dots3_loss
        return mapping_loss, dots1_loss, dots3_loss

In [ ]:
def Print_precision(S,Test,k=[100,200]):
  v = []
  for l in k:
    value = Precision(S,Test,l)
    print(f"Precision@{l}: {value}")
    v.append(v)
  return v

def find_zero(A):
  Indices = np.where(A==1)
  S = np.vstack((Indices[0],Indices[1]))
  return S.T

def Precision(S,Test,L):
  edge_index_temp = sp.coo_matrix(S)
  values = edge_index_temp.data
  indices = np.vstack((edge_index_temp.row, edge_index_temp.col,values))
  S = list(map(list, zip(*indices)))
  SS = sorted(S,key=lambda x:x[2],reverse=True)[:L]
  Top_L_indices = np.array(SS)[:,0:2]
  Test_indices = (find_zero(Test)).tolist()
  precision = 0
  for i in range(np.size(Top_L_indices,0)):
    if(Top_L_indices[i].tolist() in Test_indices):
      precision +=1
  return precision/L

def processing(S,Train):
  rows, cols = np.where(Train == 1)
  for i in range(len(rows)):
    S[rows[i]][cols[i]]=0
  return S
  
def AA(matrix):
  S = np.empty_like(matrix)
  degree = matrix.sum(axis=1)
  for i in range(len(matrix)):
    for j in range(len(matrix)):
      neighbor = [1 if (matrix[i][k]==1 and matrix[j][k]==1) else 0 for k in range(len(matrix))]
      neighbor_index = [i for i,x, in enumerate(neighbor) if x==1]
      AA_value = sum([1/math.log(degree[m]) if degree[m]>1 else 0 for m in range(len(neighbor_index))])
      S[i][j] = AA_value
  return S

def LP(matrix,sigma):
  S = matrixPow(matrix,2) + sigma*matrixPow(matrix,3)
  return S

def matrixPow(Matrix,n):
    if(type(Matrix)==list):
        Matrix=np.array(Matrix)
    if(n==1):
        return Matrix
    else:
        return np.matmul(Matrix,matrixPow(Matrix,n-1))

In [ ]:
def Get_edge_pyg(A):
  edge_index_temp = sp.coo_matrix(A)
  values = edge_index_temp.data
  indices = np.vstack((edge_index_temp.row, edge_index_temp.col))
  # edge_index_A = torch.LongTensor(indices)
  edge = torch.LongTensor(indices)
  edge_attribute = torch.FloatTensor(values)
  edge_index = torch.sparse_coo_tensor(edge, edge_attribute, edge_index_temp.shape)
  return edge_index, edge, edge_attribute

def graph_info(graph):
  print("Graph Information:\nnum_nodes:"+str(graph.num_nodes)+"  num_edges:"+str(int(graph.num_edges/2))+"  is_directed:"+str(graph.is_directed())+"\nnode_att_num:"+str(graph.num_node_features)+"  edge_att_num:"+str(graph.num_edge_features)+"  isolated_nodes:"+str(graph.has_isolated_nodes()))
  return

def One_hot_label(label):
  One_hot = np.zeros((len(label), np.max(label)-np.min(label)+1 ))
  for i in range(len(label)):
      One_hot[i][label[i]]=1
  return One_hot

def load_mat(feature,name):
    mat = feature[str(name)][:]
    mat = np.transpose(mat)
    mat = torch.tensor(mat)
    return mat

def Compute_accruracy(out,Label,test_set):
  pred = out[test_set].argmax(axis=1)
  true = Label[test_set].argmax(axis=1)
  correct = int((pred.numpy()==true).sum())
  acc = correct / len(test_set)
  return acc

def greedy_match(S):
    """
    :param S: Scores matrix, shape MxN where M is the number of source nodes,
        N is the number of target nodes.
    :return: A dict, map from source to list of targets.
    """
    S = S.T
    m, n = S.shape
    x = S.T.flatten()
    min_size = min([m,n])
    used_rows = np.zeros((m))
    used_cols = np.zeros((n))
    max_list = np.zeros((min_size))
    row = np.zeros((min_size))  # target indexes
    col = np.zeros((min_size))  # source indexes
    ix = np.argsort(-x) + 1
    #ix = np.argsort(-x) + 1
    matched = 1
    index = 1
    while(matched <= min_size):
        ipos = ix[index-1]
        jc = int(np.ceil(ipos/m))
        ic = ipos - (jc-1)*m
        if ic == 0:
            ic = 1
            continue
        if (used_rows[ic-1] == 0 and used_cols[jc-1] == 0):
            row[matched-1] = ic - 1
            col[matched-1] = jc - 1
            max_list[matched-1] = x[index-1]
            used_rows[ic-1] = 1
            used_cols[jc-1] = 1
            matched += 1
        index += 1

    result = np.zeros(S.T.shape)
    for i in range(len(row)):
        result[int(col[i]), int(row[i])] = 1
    return result

def get_statistics(alignment_matrix, groundtruth_matrix):
    pred = greedy_match(alignment_matrix)
    greedy_match_acc = compute_accuracy(pred, groundtruth_matrix)
    # print("Accuracy: %.4f" % greedy_match_acc)

    #f1_score(pred, groundtruth_matrix, labels=[0, 1], pos_label=1, average='micro')
    #MAP, AUC, Hit = compute_MAP_AUC_Hit(alignment_matrix, groundtruth_matrix)

    #print("MAP: %.4f" % MAP)
    # print("AUC: %.4f" % AUC)
    #print("Hit-precision: %.4f" % Hit)

    pred_k(alignment_matrix,groundtruth_matrix,1)
    pred_k(alignment_matrix,groundtruth_matrix,5)
    pred_k(alignment_matrix,groundtruth_matrix,10)
    pred_k(alignment_matrix,groundtruth_matrix,20)
    pred_k(alignment_matrix,groundtruth_matrix,30)
    pred_k(alignment_matrix,groundtruth_matrix,40)
    pred_k(alignment_matrix,groundtruth_matrix,100)

def compute_precision_k(top_k_matrix, gt):
    n_matched = 0
    gt_candidates = np.argmax(gt, axis = 1)
    for i in range(gt.shape[0]):
        if gt[i][gt_candidates[i]] == 1 and top_k_matrix[i][gt_candidates[i]] == 1:
            n_matched += 1
    n_nodes = (gt==1).sum()
    return n_matched/n_nodes

def compute_accuracy(greedy_matched, gt):
    n_matched = 0
    n_nodes = 0
    #print(len(greedy_matched[1]))
    #print(len(gt[1]))
    for i in range(greedy_matched.shape[0]):
        if greedy_matched[i].sum() > 0 and np.array_equal(greedy_matched[i], gt[i]):
            n_matched += 1
    # for i in range(gt.shape[0]):
    #     if gt[i].sum()==1:
    #         n_nodes +=1
    n_nodes = (gt==1).sum()
    return n_matched/n_nodes

def compute_MAP_AUC_Hit(alignment_matrix, gt):
    S_argsort = alignment_matrix.argsort(axis=1)[:, ::-1]
    m = gt.shape[1] - 1
    MAP = 0
    AUC = 0
    Hit = 0
    for i in range(len(S_argsort)):
        predicted_source_to_target = S_argsort[i]
        # true_source_to_target = gt[i]
        for j in range(gt.shape[1]):
            if gt[i, j] == 1:
                for k in range(len(predicted_source_to_target)):
                    if predicted_source_to_target[k] == j:
                        ra = k + 1
                        MAP += 1/ra
                        AUC += (m+1-ra)/m
                        Hit += (m+2-ra)/(m+1)
                        break
                break
    n_nodes = (gt==1).sum()
    MAP /= n_nodes
    AUC /= n_nodes
    Hit /= n_nodes
    return MAP, AUC, Hit

def pred_k(alignment_matrix,groundtruth_matrix,k):
    pred = top_k(alignment_matrix,k)
    # print(top_k(alignment_matrix,k))/
    pred_k = compute_precision_k(pred, groundtruth_matrix)
    print("Precision_"+str(k)+": %.4f" % pred_k)

def top_k(S, k=1):
    """
    S: scores, numpy array of shape (M, N) where M is the number of source nodes,
        N is the number of target nodes
    k: number of predicted elements to return
    """
    top = np.argsort(-S)[:,:k]
    # print(top)
    result = np.zeros(S.shape)
    for idx, target_elms in enumerate(top):
        for elm in target_elms:
            result[idx,elm] = 1
    return result

def LP_AUC(S,num_nodes,test_link):
  num_edge = np.size(test_link,1)
  negative_deges = negative_sampling(edge_index=test_link,num_nodes=num_nodes,num_neg_samples=num_edge)
  pos_pro = torch.sum(S[test_link[0]]*S[test_link[1]],dim=1)
  neg_pro = torch.sum(S[negative_deges[0]]*S[negative_deges[1]],dim=1)
  pro = pos_pro-neg_pro
  TP = len([i for i  in pro if i >=0])
  FP = len(pro)-TP
  print(f"AUC:{(TP*1)/(TP+FP)}")
  return (TP*1)/(TP+FP)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

path = './cora_01.mat'
epochs = 200
learning_rate = 0.0001
import scipy.io
feature = scipy.io.loadmat(path)
seed = 1234
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)

torch.set_default_tensor_type(torch.DoubleTensor)
feature = h5py.File(path)
# feature = scipy.io.laodmat(path)
# train_matrix = load_mat(feature,'Train')
# test_gt_matrix = load_mat(feature,'Test')
train_matrix = load_mat(feature,'cora_train_01')
test_gt_matrix = load_mat(feature,'cora_test_01')

_,edge_train,_ = Get_edge_pyg(train_matrix)
_,edge_test,_ = Get_edge_pyg(test_gt_matrix)
graph = Data(x=train_matrix, edge_index = edge_train).to(device)

In [ ]:
#TODO:Two layer GCN
torch.set_default_tensor_type(torch.DoubleTensor)
epochs = 200
learning_rate = 0.001
model = My_GCN(graph=graph).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=5e-4)
# scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer,milestones=[1000,1500,2000],gamma=0.1)

for epoch in range(epochs):
    optimizer.zero_grad()
    out = model.forward()
    loss,true_loss,neg_loss = model.loss(out)
    print(f'epoch:{epoch:},Emd_loss={loss:.3},true_loss={true_loss:.3},neg_loss={neg_loss:.3}')
    loss.backward()
    optimizer.step()
    # scheduler.step()
S = model.forward()
S = torch.matmul(S,S.t())
S = S.detach().cpu().numpy()
get_statistics(S,test_gt_matrix)
LP_AUC(torch.tensor(S),graph.num_nodes,edge_test)

In [ ]:
epochs = 200
learning_rate = 0.001

M = ['cora_M32']

for name in M:
  M = load_mat(feature,name)
  _,edge_M_AW,M_att_AW = Get_edge_pyg(M+train_matrix)
  graph_M_AW = Data(x=(M+train_matrix), edge_index = edge_M_AW, edge_attr = M_att_AW).to(device)
  model = MD_GCN(sim_graph=graph,graph=graph_M_AW,dropout=0).to(device)
  optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=5e-4)
  for epoch in range(epochs):
    optimizer.zero_grad()
    out = model.forward()
    loss,true_loss,neg_loss = model.loss(out)
    print(f'epoch:{epoch:},Emd_loss={loss:.3},true_loss={true_loss:.3},neg_loss={neg_loss:.3}')
    loss.backward()
    optimizer.step()
    del out,loss,true_loss,neg_loss
    torch.cuda.empty_cache()

  S = model.forward()
  S = torch.matmul(S,S.t())
  S = S.detach().cpu().numpy()
  print("\n"+name+"DGCN motif")
  get_statistics(S,test_gt_matrix)
  LP_AUC(torch.tensor(S),graph.num_nodes,edge_test)